In [77]:
def read_data_from_file(file_path):
    with open(file_path, 'r') as file:
        # Считываем X-вектор (первая строка)
        x_vector = list(map(int, file.readline().strip().split()))
        
        # Считываем Y-вектор (вторая строка)
        y_vector = list(map(int, file.readline().strip().split()))
        
        # Считываем количество узлов интерполирования (третья строка)
        n = int(file.readline().strip())
    
    return x_vector, y_vector, n

# Пример использования
file_path = 'untitled.txt'  # Замените на ваш путь к файлу
X, Y, n = read_data_from_file(file_path)

print("X-вектор:", X)
print("Y-вектор:", Y)
print("Количество узлов интерполирования:", n)

X-вектор: [0, 2, 3, 5, 6, -1]
Y-вектор: [-1, 113, 381, 1754, 3029, -16]
Количество узлов интерполирования: 4


In [79]:
ier = -1

# Проверка на неубывание вектора X
if not np.all(np.diff(X) <= 0):
    ier = 3

# Проверка на совпадение размеров векторов X и Y
if len(X) != len(Y):
    ier = 2

# Если ier равен 3, сортируем X и переставляем элементы в Y
#if ier == 3:
    #sorted_indices = np.argsort(X)
   # X = X[sorted_indices]
   # Y = Y[sorted_indices]

print("X:", X)
print("Y:", Y)
print("ier:", ier)

X: [0, 2, 3, 5, 6, -1]
Y: [-1, 113, 381, 1754, 3029, -16]
ier: 3


In [85]:
import numpy as np
import pandas as pd

def divided_differences(X, Y):
    n = len(X)
    # Создаем таблицу для разностных сумм
    table = np.zeros((n, n))
    table[:, 0] = Y  # Заполняем первый столбец значениями Y

    # Вычисляем разностные суммы
    for j in range(1, n):
        for i in range(n - j):
            table[i, j] = (table[i + 1, j - 1] - table[i, j - 1]) / (X[i + j] - X[i])

    return table

# Вычисляем разностные суммы
table = divided_differences(X, Y)

# Создаем DataFrame для удобного отображения
df = pd.DataFrame(table, columns=[f'Δ^{i}' for i in range(len(X))])
df.index = [f'X{i+1}' for i in range(len(X))]
print(df)


       Δ^0     Δ^1         Δ^2        Δ^3       Δ^4       Δ^5
X1    -1.0    57.0   70.333333  13.833333  0.055556  0.013889
X2   113.0   268.0  139.500000  14.166667  0.041667  0.000000
X3   381.0   686.5  196.166667  14.041667  0.000000  0.000000
X4  1754.0  1275.0  140.000000   0.000000  0.000000  0.000000
X5  3029.0   435.0    0.000000   0.000000  0.000000  0.000000
X6   -16.0     0.0    0.000000   0.000000  0.000000  0.000000


In [87]:
# Проверяем значение в последнем столбце первой строки
if table[0][-1] == 0:
    ier = 1  # Если значение равно 0
else:
    ier = 0  # Если значение не равно 0

print("ier =", ier)
# Индекс элемента, который будем исключать
index_to_exclude = 0
# Функция для вычисления разностных сумм
def divided_differences(X, Y):
    n = len(Y)
    table = np.zeros((n, n))
    table[:, 0] = Y

    for j in range(1, n):
        for i in range(n - j):
            table[i][j] = (table[i + 1][j - 1] - table[i][j - 1]) / (X[i + j] - X[i])
    
    return table

# Цикл проверки
while ier == 0 and index_to_exclude < len(X):
    # Создаем новые векторы без элемента с индексом index_to_exclude
    new_X = np.delete(X, index_to_exclude)
    new_Y = np.delete(Y, index_to_exclude)
    # Строим таблицу разностных сумм
    table = divided_differences(new_X, new_Y)
    # Проверяем первую строку последнего столбца
    if table[0][-1] == 0:
        print(f"Номер строки с ошибкой: {index_to_exclude+1}")
        ier = 1  # Завершаем цикл, если условие выполнено
    # Увеличиваем индекс для следующей итерации
    index_to_exclude += 1


ier = 0
Номер строки с ошибкой: 3
Цикл завершен. ier = 1
